In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import tensorflow as tf

In [2]:
data = pd.read_csv('./PopcornSmallProphetInputDF.csv')
data.ds = pd.DatetimeIndex(data.ds)
changepoints = pkl.load(open('changepoints.pkl','rb'),encoding='latin').index
changepoints

Int64Index([ 30,  60,  90, 120, 150, 180, 211, 241, 271, 301, 331, 361, 391,
            421, 451, 481, 511, 541, 572, 602, 632, 662, 692, 722, 752],
           dtype='int64')

# OMIT


In [3]:
split_data = []
split_data.append(data[data.index < changepoints[0]].y.values)
for i in range(len(changepoints)-1):
    temp = data[(data.index >= changepoints[i]) & (data.index < changepoints[i+1])].y.values
    split_data.append(temp)
split_data.append(data[data.index > changepoints[-1]].y.values)
len(split_data)

26

In [4]:
for i in split_data:
    print (np.max(i), np.min(i))
    print (np.log(np.max(i)),np.log(np.min(i)), np.mean(np.log(i)))

667 211
6.50279004592 5.35185813348 6.00787310706
613 212
6.41836493594 5.35658627467 5.88582421294
601 241
6.39859493454 5.48479693349 5.9257064235
672 328
6.51025834052 5.79301360838 6.17537304493
708 312
6.56244409369 5.74300318781 6.15116453572
714 290
6.57088296234 5.66988092298 6.05270141301
753 284
6.6240652278 5.64897423816 6.061858176
663 242
6.49677499019 5.48893772616 6.00485166154
670 218
6.50727771239 5.38449506279 5.99908713014
632 211
6.44888939415 5.35185813348 5.93333965407
603 242
6.40191719673 5.48893772616 5.83954720685
794 315
6.67708346125 5.75257263883 6.31991027217
697 344
6.54678541076 5.84064165737 6.17376094104
638 274
6.45833828334 5.61312810639 6.0461965605
674 216
6.51323011091 5.37527840768 5.97151616209
719 229
6.57786135772 5.43372200355 6.00374714285
644 209
6.4676987261 5.34233425196 5.88050110263
639 231
6.45990445438 5.44241771052 5.92033362558
601 213
6.39859493454 5.36129216571 5.9225474384
595 157
6.38856140555 5.05624580535 5.74602424302
668 223

In [5]:
a = np.log(np.max(split_data[0]))
b = np.log(np.min(split_data[0]))

for i in split_data[1:]:
    temp1 = np.power(np.max(i),1/a)
    temp2 = np.power(np.min(i),1/b)
    temp = (temp1 + temp2)/2
    temp = (np.log(i)/np.log(temp))
    print (np.max(temp),np.min(temp))

6.45727446756 5.3890590719
6.36918832183 5.45958994651
6.24416318809 5.55623455008
6.30010619407 5.51342296246
6.34716686129 5.47684086113
6.38616178689 5.44609121049
6.41699718417 5.42153575982
6.485241935 5.36626139584
6.4756717164 5.37408447051
6.36840658587 5.46020607429
6.3523076918 5.47276541223
6.2353824963 5.56282701701
6.32315861524 5.4956395573
6.49380229326 5.35924489943
6.49074151615 5.36175560379
6.49097739839 5.36156248964
6.42628073075 5.41408999295
6.44428912556 5.39957868187
6.62867463593 5.24628413117
6.47002512617 5.37868807308
6.39930049148 5.43568688172
6.53356421988 5.32641342208
6.37661056598 5.45376624705
6.37563107118 5.45452591473
7.09259947822 4.82337533913


# Trying

In [3]:
learning_rate = 0.001
training_iters = 100
batch_size = 32
display_step = 10

n_input = 10
dropout = 0.5

x = tf.placeholder(tf.float32, shape=(None, 10, 4))
y = tf.placeholder(tf.float32, shape=(None, 1))

filters = 5
inputChannels = 4
outputChannels = 300

W_conv1 = tf.Variable(tf.truncated_normal([filters, inputChannels, outputChannels],stddev=0.1))
b_conv1 = tf.Variable(tf.constant(.1,shape=[outputChannels]))

print (b_conv1)
h_conv1 = tf.nn.conv1d(x, filters=W_conv1, stride= 1, padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(h_conv1)
print (h_conv1)
h_pool1 = tf.layers.max_pooling1d(h_conv1, pool_size=[3], strides= 1, padding='SAME')
print (h_pool1)

<tf.Variable 'Variable_3:0' shape=(300,) dtype=float32_ref>
Tensor("Relu_1:0", shape=(?, 10, 300), dtype=float32)
Tensor("max_pooling1d_1/Squeeze:0", shape=(?, 10, 300), dtype=float32)
